# Imports

In [ ]:
import jax
import jax.numpy as jnp
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import sparse as spa

from eikonax import (
    derivator,
    finitediff,
    logging,
    preprocessing,
    solver,
    tensorfield,
)

sns.set_theme(style="ticks")

# Configuration and setup

In [ ]:
# Jax
disable_jit = False
log_compiles = False

# Mesh
mesh_bounds_x = (0, 1)
mesh_bounds_y = (0, 1)
num_points_x = 3
num_points_y = 3

# Solver
solver_data = solver.SolverData(
    tolerance=1e-8,
    max_num_iterations=1000,
    loop_type="jitted_while",
    max_value=1000,
    use_soft_update=True,
    softminmax_order=100,
    softminmax_cutoff=0.01,
    log_interval=1,
)

# Derivator
derivator_data = derivator.PartialDerivatorData(
    use_soft_update=True,
    softminmax_order=100,
    softminmax_cutoff=0.01,
)

# Logger (optional)
logger_data = logging.LoggerSettings(
    log_to_console=True,
    logfile_path=None,
)

# Initial sites
initial_sites = preprocessing.InitialSites(inds=(0,), values=(0,))

In [ ]:
# Create Mesh
vertices, simplices = preprocessing.create_test_mesh(
    mesh_bounds_x, mesh_bounds_y, num_points_x, num_points_y
)
mesh_data = preprocessing.MeshData(vertices, simplices)

# Initialize tensor field
rng = np.random.default_rng(seed=0)
parameter_vector = rng.uniform(0.5, 1.5, simplices.shape[0])
parameter_vector = jnp.array(parameter_vector)

tensor_on_simplex = tensorfield.InvLinearScalarSimplexTensor(dimension=vertices.shape[1])
tensor_field_mapping = tensorfield.LinearScalarMap()
tensor_field_object = tensorfield.TensorField(
    num_simplices=simplices.shape[0],
    vector_to_simplices_map=tensor_field_mapping,
    simplex_tensor=tensor_on_simplex,
)

# Initialize logger
logger = logging.Logger(logger_data)

# Forward solver

In [ ]:
with jax.disable_jit(disable_jit), jax.log_compiles(log_compiles):
    tensor_field_instance = tensor_field_object.assemble_field(parameter_vector)
    eikonal_solver = solver.Solver(mesh_data, solver_data, initial_sites, logger)
    solution = eikonal_solver.run(tensor_field_instance)

# Parametric derivatives

In [ ]:
with jax.disable_jit(disable_jit), jax.log_compiles(log_compiles):
    # Compute partial derivatives
    eikonax_derivator = derivator.PartialDerivator(mesh_data, derivator_data, initial_sites)
    output_partial_solution, output_partial_tensor = eikonax_derivator.compute_partial_derivatives(
        solution.values, tensor_field_instance
    )
    tensor_partial_parameter = tensor_field_object.assemble_jacobian(parameter_vector)
    output_partial_parameter = spa.einsum(
        "ijkl,jklm->im", output_partial_tensor, tensor_partial_parameter
    )
    # Initialize derivative solver
    derivative_solver = derivator.DerivativeSolver(solution.values, output_partial_solution)

In [ ]:
# Compute Jacobian using Eikonax
eikonax_jacobian = derivator.compute_eikonax_jacobian(derivative_solver, output_partial_parameter)

# Compute Jacobian using finite differences
step_widths = np.logspace(-5, -1, 101)
errors = []
for step_width in step_widths:
    finite_diff_jacobian = finitediff.compute_fd_jacobian(
        eikonax_solver=eikonal_solver,
        tensor_field=tensor_field_object,
        stencil=finitediff.finite_diff_1_forward,
        eval_point=parameter_vector,
        step_width=step_width,
    )
    error = np.linalg.norm(finite_diff_jacobian - eikonax_jacobian)
    errors.append(error)

# Plot difference
fig, ax = plt.subplots(figsize=(5, 5), layout="constrained")
ax.set_xlabel(r"$h$")
ax.set_ylabel(r"$\|J_{\text{FD}} - J_{\text{Eikonax}}\|_2$")
ax.loglog(step_widths, errors)